# A Mystery in Fiftyville 🕵️

Write SQL queries to solve a mystery.<br>
*[problem 7 from Harvard CS50](https://cs50.harvard.edu/x/2021/psets/7/)* 

## The Sotry 🐥

The CS50 Duck has been stolen! The town of Fiftyville has called upon you to solve the mystery of the stolen duck. Authorities believe that the thief stole the duck and then, shortly afterwards, took a flight out of town with the help of an accomplice. 
<br>
<br>

## Your goal is to identify 🎯

- Who the thief is ?
- What city the thief escaped to ?
- Who the thief’s accomplice ?
<br>
<br>


## Clues 🤔

All you know is that the theft **took place on July 28, 2020** and that it **took place on Chamberlin Street**.

In [1]:
from cs50 import SQL

# Let's connect to database with sqlite3
database = SQL("sqlite:///fiftyville.db")

The database is compose by 9 tables:
- `airports`                  
- `crime_scene_reports`       
- `people`
- `atm_transactions`          
- `flights`                   
- `phone_calls`
- `bank_accounts`             
- `interviews`
- `courthouse_security_logs`  
- `passengers`

Let's first access the theft description through the `crime_scene_reports` table. 

In [2]:
database.execute(
    """
    SELECT description 
    FROM crime_scene_reports 
    WHERE 
        year = 2020 AND 
        month = 07 AND 
        day =28 AND 
        street = "Chamberlin Street"
    """
)

[{'description': 'Theft of the CS50 duck took place at 10:15am at the Chamberlin Street courthouse. Interviews were conducted today with three witnesses who were present at the time — each of their interview transcripts mentions the courthouse.'}]

Great ! We know that the theft took place at `10:15am` at `the Chamberlin Street courthouse` and there are `3 witnesses`. 

Let's take a look at the interviews

In [3]:
database.execute(
    """
    SELECT name,transcript 
    FROM interviews 
    WHERE 
        year = 2020 AND 
        month = 07 AND 
        day = 28;
    """
)

[{'name': 'Jose',
  'transcript': '“Ah,” said he, “I forgot that I had not seen you for some weeks. It is a little souvenir from the King of Bohemia in return for my assistance in the case of the Irene Adler papers.”'},
 {'name': 'Eugene',
  'transcript': '“I suppose,” said Holmes, “that when Mr. Windibank came back from France he was very annoyed at your having gone to the ball.”'},
 {'name': 'Barbara',
  'transcript': '“You had my note?” he asked with a deep harsh voice and a strongly marked German accent. “I told you that I would call.” He looked from one to the other of us, as if uncertain which to address.'},
 {'name': 'Ruth',
  'transcript': 'Sometime within ten minutes of the theft, I saw the thief get into a car in the courthouse parking lot and drive away. If you have security footage from the courthouse parking lot, you might want to look for cars that left the parking lot in that time frame.'},
 {'name': 'Eugene',
  'transcript': "I don't know the thief's name, but it was so

## 3 INTERVIEWS
 
"Sometime within ten minutes of the theft, **I saw the thief get into a car in the courthouse parking lot and drive away**.
If you have security footage from the courthouse parking lot, you might want to look for cars that left the parking lot in that time frame." *- RUTH* 👩‍🦳

"I don't know the thief's name, but it was someone I recognized. Earlier this morning, before I arrived at the courthouse,
**I was walking by the ATM on Fifer Street and saw the thief there withdrawing some money.**" *- EUGENE* 👱

"As the thief was leaving the courthouse, they called someone who talked to them for less than a minute.
In the call, I heard the thief say that they were planning to take the earliest flight out of Fiftyville tomorrow. The thief then asked the person on the other end of the phone to purchase the flight ticket." *- RAYMOND* 🧓

## 4 clues 🧐
- The thief withdrew money from the Fifer Street ATM before the robbery.
- The thief left the courthouse parking lot within 10 minutes of the robbery.
- When the thief left the courthouse, we called someone for less than a minute.
- The thief and his acolyte got on the first flight on July 29, 2020.

By processing all these clues and joining the information, we will find the thief, his sidekick and the place of their escape.

## Summary
1. PARKING LOT INFO
2. ATM INFO
3. FLIGHT INFO
4. PHONE INFO

## 1. PARKING LOT INFO
*table used: `courthouse_security_logs` & `people`*
### Which Licence left the courthouse between 10 & 11 am ?

In [4]:
# Take all of the licence who left the courthouse parking between 10 & 11 am
suspect_licence_plate = (
"""
    SELECT license_plate 
    FROM courthouse_security_logs  
    WHERE 
        activity = "exit" AND 
        year = 2020 AND 
        month = 07 AND 
        day =28 AND 
        hour BETWEEN 10  AND  11 AND
        minute<25 
"""
)
database.execute(suspect_licence_plate)

[{'license_plate': '5P2BI95'},
 {'license_plate': '94KL13X'},
 {'license_plate': '6P58WS2'},
 {'license_plate': '4328GD8'},
 {'license_plate': 'G412CB7'},
 {'license_plate': 'L93JTIZ'},
 {'license_plate': '322W7JE'},
 {'license_plate': '0NTHK55'}]

### Get the name of the potential suspect form there license_plate

In [5]:
suspect_driver = (
f"""
    SELECT name 
    FROM people 
    WHERE license_plate IN({suspect_licence_plate})
""")
database.execute(suspect_driver)

[{'name': 'Patrick'},
 {'name': 'Amber'},
 {'name': 'Elizabeth'},
 {'name': 'Roger'},
 {'name': 'Danielle'},
 {'name': 'Russell'},
 {'name': 'Evelyn'},
 {'name': 'Ernest'}]

Great ! We know that the thieft is inside this 9 names. Let's now take a look at the people who was using the atm on Fifer street this days 

## 2. ATM INFO
*table used: `atm_transactions`, `bank_accounts` & `people`*
### Which account numbers have been withdrawn this day in Fifer Street? 

In [6]:
suspect_account_number =(
"""
    SELECT account_number  
    FROM atm_transactions
    WHERE 
        year = 2020 AND 
        month = 07 AND 
        day =28 AND 
        transaction_type='withdraw' 
        AND atm_location = "Fifer Street"
"""
)
database.execute(suspect_account_number)

[{'account_number': 28500762},
 {'account_number': 28296815},
 {'account_number': 76054385},
 {'account_number': 49610011},
 {'account_number': 16153065},
 {'account_number': 25506511},
 {'account_number': 81061156},
 {'account_number': 26013199}]

### Get the person id from the account number 

In [7]:
suspect_bank_id = (
f"""
    SELECT person_id 
    FROM bank_accounts 
    WHERE account_number IN({suspect_account_number})

"""
)
database.execute(suspect_bank_id)

[{'person_id': 686048},
 {'person_id': 514354},
 {'person_id': 458378},
 {'person_id': 395717},
 {'person_id': 396669},
 {'person_id': 467400},
 {'person_id': 449774},
 {'person_id': 438727}]

### Get the name of people from there bank id

In [8]:
suspect_from_atm = (
    f"""
    SELECT name 
    FROM people 
    WHERE id IN({suspect_bank_id})
    """
)
database.execute(suspect_from_atm)

[{'name': 'Bobby'},
 {'name': 'Elizabeth'},
 {'name': 'Victoria'},
 {'name': 'Madison'},
 {'name': 'Roy'},
 {'name': 'Danielle'},
 {'name': 'Russell'},
 {'name': 'Ernest'}]

Nice ! In those 8 names we have the thief. Let's now investigate on flight information 

## 3 . FLIGHT INFO
*table used: `flights`, `airports`, `passanger` & `people`*
### What is the first flight from Fiftyville on July 29, 2020?

In [9]:
def get_flight_info(col='*'):
    return (
        f"""
        SELECT flights.{col}
        FROM flights 
        JOIN airports ON flights.origin_airport_id = airports.id
        WHERE 
            year = 2020 AND
            month = 07 AND 
            day =29 AND
            city = "Fiftyville"
        ORDER BY hour
        LIMIT 1
        """
    )
all_flight_info = get_flight_info(col='*')
database.execute(all_flight_info)


[{'id': 36,
  'origin_airport_id': 8,
  'destination_airport_id': 4,
  'year': 2020,
  'month': 7,
  'day': 29,
  'hour': 8,
  'minute': 20}]

The first flight from Fiftyville on the 29th of July, with `id 6`, going to `destination_airport_id': 4` leaved at 8.20 am. 

### Who was the passenger of this flight ?

In [10]:
first_flight_id = get_flight_info("id")

suspect_passengers = (
    f"""
    SELECT people.name FROM passengers
    JOIN people 
    ON people.passport_number = passengers.passport_number
    WHERE flight_id = ({first_flight_id})
    """
)
database.execute(suspect_passengers)

[{'name': 'Doris'},
 {'name': 'Roger'},
 {'name': 'Ernest'},
 {'name': 'Edward'},
 {'name': 'Evelyn'},
 {'name': 'Madison'},
 {'name': 'Bobby'},
 {'name': 'Danielle'}]

We are lucky that the flight was almost empty! The thief and his acolyte are among these 8 people. Now let's find out where they were escaping to

### Where the flight going to ?

In [11]:
destination_airport_id = get_flight_info("destination_airport_id")
flight_destination = (
    f"""
    SELECT * 
    FROM airports 
    WHERE id = ({destination_airport_id})
    """
)
database.execute(flight_destination)

[{'id': 4,
  'abbreviation': 'LHR',
  'full_name': 'Heathrow Airport',
  'city': 'London'}]

#### They were escaping to London 💂‍♀️

## 4. PHONE CALL
*table used: `phone_number` & `people`*
### What were the calls under the minute this day? 

In [12]:
suspect_from_phone_call = (
    """
    SELECT name FROM people 
    WHERE phone_number IN (
        SELECT caller 
        FROM phone_calls 
        WHERE  
            year = 2020 AND 
            month = 07 AND 
            day = 28 AND 
            DURATION < 60
    )
    """
)
database.execute(suspect_from_phone_call)

[{'name': 'Bobby'},
 {'name': 'Roger'},
 {'name': 'Victoria'},
 {'name': 'Madison'},
 {'name': 'Russell'},
 {'name': 'Evelyn'},
 {'name': 'Ernest'},
 {'name': 'Kimberly'}]

Great! We know that the thief is among these people. Now Let's mix all together to find the thief !

# Result 😎

## Who is the thief ?

In [13]:
# Group All in one query
find_the_thief = (
    f"""
    SELECT name 
    FROM people 
    WHERE 
        name IN ({suspect_driver}) AND
        name IN ({suspect_from_atm}) AND
        name IN ({suspect_passengers}) AND
        name IN ({suspect_from_phone_call})
    """
) 
database.execute(find_the_thief)


[{'name': 'Ernest'}]

Ernest is the thief 🥷

## Who is the acolyte ?

In [14]:
# get the acolyte number
acolyte_number = (
    f"""
    SELECT phone_calls.receiver
    FROM phone_calls
    JOIN people ON people.phone_number = phone_calls.caller
    WHERE  
        year = 2020 AND
        month = 07 AND 
        day = 28 AND 
        duration < 60 AND 
        name = "Ernest"
    """
)
database.execute(acolyte_number)

[{'receiver': '(375) 555-8161'}]

In [15]:
# get the acolyte name
find_the_acolyte = (
    f"""
    SELECT name
    FROM people
    WHERE phone_number = ({acolyte_number})
    """
)
database.execute(find_the_acolyte)

[{'name': 'Berthold'}]

Berthold is the acolyte 😈

<div class="alert alert-block alert-info">
<h2>summary:</h2>
<ul>
  <li>Ernest is the thief</li>
  <li>Berthold the acolyte</li>
  <li>they escaped to London</li>
</ul>
</div>